# Solving continuous optimization problems.
Traditionally, optimization problems are grouped in two distinct natural categories: those with continuous variables and those with discrete variables. When solving an instance of a continuous optimization problem, one is generally interested in finding a set of real numbers (possibly parameterizing a function). 

Continuous problems can be classified in two categories: unconstrained and constrained. The unconstrained continuous problems do not impose any explicit spatial constraints on the candidate solution’s validity. In practice, however, the underlying data types bound the solution representations. Contrarily, constrained problems do impose explicit spatial constraints on the solution’s validity. Following the mathematical definition, the constraints can be either hard, as they set explicit conditions for the solutions that are required to be satisfied, or soft, as they set conditions that penalize the fitness function if they are not satisfied.

## Box problem type.
We have conceptualized a module called *continuous* that contains different problem types from the continuous optimization field. In this release, the module contains one class called ``Box``: a simplistic variant of a constrained continuous problem in which the parameters can take any real number within a given range of values, the box (a.k.a. hyperrectangle), which can be regular case when the bounds are the same for every dimension or irregular when each dimension is allowed to have different bounds.

### Rastrigin function.
Rastrigin function is a non-convex, highly multimodal function used as a performance test problem for optimization algorithms. The locations of the minima are regularly distributed. Formally:
$$ f(x) = 10d+\sum_{i=1}^{d}[x_i^2-10cos(2 \pi x_i)], $$
where $d$ is the number of problem's dimensions. Visually, for $d=2$:

<img src="https://www.sfu.ca/~ssurjano/rastr.png" alt="Drawing" style="width: 400px;"/>

# 1. Create problem's instance.

Loads the necessary classes and functions.

In [38]:
# Imports PyTorch
import torch
# Imports problems
from gpol.problems.continuous import Box
from gpol.problems.utils import rastrigin_function
# Imports metaheuristics 
from gpol.algorithms.random_search import RandomSearch
from gpol.algorithms.local_search import HillClimbing, SimulatedAnnealing
from gpol.algorithms.genetic_algorithm import GeneticAlgorithm
from gpol.algorithms.differential_evolution import DifferentialEvolution
from gpol.algorithms.swarm_intelligence import SPSO, APSO
# Imports operators
from gpol.operators.initializers import rnd_muniform, rnd_vuniform
from gpol.operators.selectors import prm_dernd_selection, prm_tournament
from gpol.operators.variators import prm_pso, prm_iball_mtn, geometric_xo, de_rand, de_exponential_xo,  de_binomial_xo, de_target_to_best

Creates an instance of ``Box`` problem. The search space ($S$) of an instance of ``Box`` problem consists of the following key-value pairs:
- ``"constraints"``: is a tensor that holds the lower and the upper bounds for the search space’s box. When the box is intended to be regular (i.e., all its dimensions are equally sized), it tensor holds only two values, each representing the lower and the upper bounds of each of the $D$ dimensions of the problem, respectively. When the box is intended to be irregular, the tensor is a $2$x$D$ matrix. In such case, the first and the second row represent the lower and the upper bounds for each of the $D$ dimensions of the problem, respectively; and
-  ``"n_dims"``: an integer value representing the dimensionality ($d$) of $S$.

When solving hard-constrained continuous problems, some authors in the scientific community bound the solutions to prevent the searching in infeasible regions. The bounding mechanism generally consists of a random re-initialization of the solution on the outlying dimension. For this reason, besides the triplet ``sspace``, ``ffunction``, and ``min_``, an instance of ``Box`` problem includes an additional instance-variable called ``bound``, which can optionally activate the solutions’ bounding by assigning it the value ``True``. When ``bound=False``, the outlying solution automatically receives the *worst* fitness score (corresponding to the largest or the smallest value allowed by the underlying data type).

In [50]:
# Defines the processing device and random state 's seed
device, seed = "cpu", 0  # 'cuda' if torch.cuda.is_available() else 'cpu', 0
# Creates the search space
sspace = {"constraints": torch.tensor([-5.12, 5.12], device=device), "n_dims": 2}
# Creates an instance of Box
pi = Box(sspace=sspace, ffunction=rastrigin_function, min_=True, bound=True)

# 2. Choose and parametrize the algorithms.

## 2.1. Random search (RS).
The random search (RS) can be seen as thethe first rudimentary stochastic metaheuristic for problem-solving. Its strategy, far away from being *intelligent*, consists of randomly sampling $S$ for a given number of iterations. As such, the only search-parameter of an instance of ``RandomSearch`` is the initialization function (the ``initializer``). The function ``rnd_vuniform`` is one of the numerous initializers for the single-point (SP) algorithms and it returns a uniformly distributed vector of floats.

The cell in below creates a dictionary called ``pars`` which stores algorithms' parameters. Each key-value pair stores the algorithm's type and a dictionary of respective search-parameters. The first key-value pair regards the RS.

In [40]:
# Defines a single-point (SP) initializer
sp_init = rnd_vuniform
# Defines RS's parameters
pars = {RandomSearch: {"initializer": sp_init}}

## 2.2. Hill climbing (HC).
The local search (LS) algorithms can be seen among the first intelligent search strategies that improve the functioning of the RS. They rely upon the concept of neighborhood which is explored at each iteration by sampling from $S$ a limited number of neighbors of the best-so-far solution. Usually, the LS algorithms are divided in two branches. In the first branch, called hill climbing (HC), or hill descent for the minimization problems, the best-so-far solution is replaced by its neighbor when the latter is at least as good as the former.

The cell in below adds ``HillClimbing`` to ``pars``. Note that, unlike it was for ``RandomSearch``, an instance of ``HillClimbing`` also requires the specification of a neighbor-generation function (``"nh_function"``) and the neighborhood's size (``"nh_size"``). In this example, the so-called *ball-mutation* is used, with a probability of mutating a given index of a candidate solution equal to $100$\% (since the example is 2D), and a maximum radius of mutation equal to $0.3$.
Note that the very same initialization function is used for both ``RandomSearch`` and ``HillClimbing``. 

In [51]:
# Defines the size of the neighborhood 
nh_size = 100
# Defines neighbor-generation function with the respective parameters
p_ball_mnt, radius = 1.0, 0.3  
nh_function = prm_iball_mtn(prob=p_ball_mnt, radius=radius)
# Defines HC's parameters
pars[HillClimbing] = {"initializer": sp_init, "nh_function": nh_function, "nh_size": nh_size}

## 2.3. Simulated annealing (SA).
The second branch, called simulated annealing (SA), extends HC by formulating a non-negative probability of replacing the best-so-far solution by its neighbor when the latter is worse. Traditionally, such a probability is small and decreases as the search advances. The strategy adopted by SA is especially useful when the search is prematurely tagnated at a locally sub-optimal point in $S$.

The cell in below adds ``SimulatedAnnealing`` to ``pars``. 

In [42]:
# Defines SA's parameters
pars[SimulatedAnnealing] = {"initializer": sp_init, "nh_function": nh_function, "nh_size": nh_size, "control": 1.0, "update_rate": 0.9}

## 2.4. Genetic algorithm (GA).
Based on the number of candidate solutions they handle at each step, the metaheuristics can be categorized into single-point (SP) and population-based (PB) approaches. 
The search procedure in the SP metaheuristics is generally guided by the information provided by a single candidate solution from $S$, usually the best-so-far solution, that is gradually evolved in a well-defined manner in hope to find the global optimum. The abovementioned HC and SA are examples of SP metaheuristics as the search is performed by exploring the neighborhood $N(i)$, where $i$ is the current best solution. Contrarily, the search procedure in PB metaheuristics is generally guided by the information shared by a set of candidate solutions and the exploitation of its collective behavior in different ways. In abstract terms, one can say that every PB metaheuristics shares, at least, the following two features: an object representing the set of simultaneously exploited candidate solutions (i.e., the population), and a procedure to *move* them across $S$.

Genetic Algorithm (GAs) is a meta-heuristic introduced by J. Holland which was strongly inspired by Darwin's theory of evolution by means of natural selection. Conceptually, the algorithm starts with a random-like population of candidate solutions (called *chromosomes*). Then, by mimicking the natural selection and genetically inspired variation operators, such as the crossover and the mutation, the algorithm breeds a population of the next-generation candidate solutions (called the *offspring population*), that replaces the previous population (a.k.a. the *parent population*). This procedure is iterated until reaching some stopping criteria, like a maximum
number of iterations (also called *generations*).

The cell in below adds ``GeneticAlgorithm`` to ``pars``. It uses a slightly different initializer, specially designed to efficiently initialize PB metaheuristics, called ``rnd_muniform``, that returns a uniformly distributed matrix of floats. Note that the mutation function is the neighbor-generation function, that was used for the aforementioned LS algorithms, and the population's size is equivalent to the neighborhood's size; this is done to foster the equivalency between LS and PB metaheuristics. Finally, the crossover is geometric, the elite is always a member of the population and parents' reproduction is enabled (``elitism=True`` and ``reproduction=True``).

In [43]:
# Defines a population-based (PB) initializer
pb_init = rnd_muniform
# Defines GA's parameters
pars[GeneticAlgorithm] = {"pop_size": nh_size, "initializer": pb_init, "selector": prm_tournament(pressure=0.05), "mutator": nh_function, 
                          "crossover": geometric_xo, "p_m": 0.3, "p_c": 0.7, "elitism": True, "reproduction": True}

## 2.5. Synchronous particle swarm optimization (S-PSO).
Particle swarm optimization (PSO) is another form of PB metaheuristic, developed by Eberhart and Kennedy in 1995. Contrarily to previously presented GA, PSO was inspired by the social behavior of living organisms, such as the birds and fish, when looking for food sources. Following PSO's nomenclature, a population is called *swarm* and a candidate solution is a *particle*.

Formally, the procedure for particle's position update is defined as: 

$$\vec{x}_{(p,i)} = \vec{x}_{(p,\ i-1)} + \vec{v}_{(p,i)}, $$

such that: 

$$\vec{v}_{(p,i)} = w*\vec{v}_{(p,\ i-1)} + C_1\vec{\phi_1}(\vec{lbest}_p - \vec{x}_{(p,\ i-1)}) + C_2\vec{\phi_2}(\vec{gbest} - \vec{x}_{(p,\ i-1)}), $$ 

where $\vec{lbest}_p$ and $\vec{gbest}$ represent the local and global best, respectively, with $C_1$ and $C_2$ being two positive constants used to scale their contribution in the equation (a.k.a. acceleration coefficients). The quantities $\vec{\phi_1}$ and $\vec{\phi_2}$ are two random vectors which values follow~$\sim$ $U(0,\ 1)$ at each dimension. 

In its original definition, swarm's positions are updated taking in consideration the same version of global best particle, obtained after evaluating the whole swarm at previous iteration. That is, the global best particle is first identified and then used by all the particles in the swarm. The strength of this update method, frequently called Synchronous-PSO (S-PSO), relies in the exploitation of the information.

The cell in below adds ``SPSO`` to ``pars``. This time, the ``"mutator"`` is the function ``prm_pso`` which encapsulates PSO's update rule with parameters ``C_1``, ``C_2``, ``w_max`` and ``w_min``. 

In [44]:
# Defines SPSO's parameters
update_rule = prm_pso(c1=1.0, c2=1.0, w_max=0.9, w_min=0.4)
pars[SPSO] = {"pop_size": nh_size, "initializer": pb_init, "mutator": update_rule}

## 2.6. Asynchronous PSO.
Alternatively, the scientific community proposed an Asynchronous update (A-PSO), where global best particle is identified immediately after updating the position of each particle in the swarm. The strength of this update method relies in the exploration of the information.

The cell in below adds ``APSO`` to ``pars``. Note that the ``"mutator"`` function is the same as in S-PSO (``prm_apso``), suggesting that the underlying update rule remains the same.

In [45]:
# Defines APSO's parameters
pars[APSO] = {"pop_size": nh_size, "initializer": pb_init, "mutator": update_rule}

## 2.7. Differential evolution (DE).
Differential evolution (DE) is another type of metaheuristic we have considered including in our library. Storn and Price originally designed this PB metaheuristic for solving continuous optimization problems in 1995. The algorithm shares many similar features with GA, as it involves maintaining a population of candidate solutions, which are exposed to iterative selection and variation (aka recombination). Nevertheless, DE differs substantially from GA in how the selection and the variation are performed. The parent selection is performed at random, meaning that all the chromosomes have an equal probability of being selected for mating, regardless of their fitness. The variation consists of two steps: mutation and crossover. Numerous different operators were proposed so far and many are included in this library. 

The cell in below adds ``DifferentialEvolution`` to ``pars``. The function ``de_rand`` implements a generalization of the the *DE/RAND/N* mutation strategy, and is formally defined as $𝑉(𝐺+1) = 𝑉(r_1, 𝐺) + 𝐹_1[𝑉(r_2, 𝐺) − V(r_3, 𝐺)] + (...) + 𝐹_N[𝑉(r_{(N*2)}, 𝐺) − V(r_{(N*2+1)}, 𝐺)]$. In this sense, *DE/RAND/N* strategy creates the *donor* vector (i.e., $𝑉(𝐺+1)$) from adding $N$ weighted differences between $2N$ randomly selected parent vectors to another $(2N +1)^{th}$ random parent. 

The ``n_sols`` parameter in ``prm_dernd_selection`` function allows to adapt the desired mutation strategy for a given value of *N*: ``n_sols==3`` results in *DE/RAND/1*.

In [46]:
# Defines DE's parameters
n_parents, m_weights = 3, torch.tensor([0.9], device=device)
pars[DifferentialEvolution] = {"pop_size": nh_size, "initializer": pb_init, "selector": prm_dernd_selection(n_sols=3), 
                               "mutator": de_rand, "crossover": de_exponential_xo, "m_weights": m_weights, "c_rate": 0.5}

# 3. Executes the experiment.

Note that *many* parameters and functions are shared across different algorithms in the experiment. This allows to increase the control and comparability between different algorithmic approaches when solving a given problem's instance.

In [47]:
for isa_type, isa_pars in pars.items():
    print(isa_type)
    for p_name, p_val in isa_pars.items():
        print("\t", p_name, p_val)

<class 'gpol.algorithms.random_search.RandomSearch'>
	 initializer <function rnd_vuniform at 0x000001B5B10A2040>
<class 'gpol.algorithms.local_search.HillClimbing'>
	 initializer <function rnd_vuniform at 0x000001B5B10A2040>
	 nh_function <function prm_iball_mtn.<locals>.iball_mnt at 0x000001B5B114DE50>
	 nh_size 100
<class 'gpol.algorithms.local_search.SimulatedAnnealing'>
	 initializer <function rnd_vuniform at 0x000001B5B10A2040>
	 nh_function <function prm_iball_mtn.<locals>.iball_mnt at 0x000001B5B114DE50>
	 nh_size 100
	 control 1.0
	 update_rate 0.9
<class 'gpol.algorithms.genetic_algorithm.GeneticAlgorithm'>
	 pop_size 100
	 initializer <function rnd_muniform at 0x000001B5B10A20D0>
	 selector <function prm_tournament.<locals>.tournament at 0x000001B5B116C040>
	 mutator <function prm_iball_mtn.<locals>.iball_mnt at 0x000001B5B114DE50>
	 crossover <function geometric_xo at 0x000001B59A3000D0>
	 p_m 0.3
	 p_c 0.7
	 elitism True
	 reproduction True
<class 'gpol.algorithms.swarm_int

Defines the computational resources for the experiment: the number of iterations.

In [48]:
n_iter = 30

Loops the afore-defined ``pars`` dictionary containing algorithms' and the underlying parameters. Note that besides algorithm-specific parameters, the constructor of an instance of a search algorithm also receives the random state to initialize a pseudorandom number generator (called ``seed``), and the specification of the processing ``device`` (either CPU or GPU).

The ``solve`` method has the same signature for all the search algorithms and, in this example, includes the following parameters: 
-  ``n_iter``: number of iterations to conduct the search;
-  ``tol``: minimum required fitness improvement for ``n_iter_tol`` consecutive iterations to continue the search. When the fitness is not improving by at least ``tol`` for ``n_iter_tol`` consecutive iterations, the search will be automatically interrupted;
-  ``n_iter_tol``: maximum number of iterations to not meet ``tol`` improvement;
-  ``verbose``: verbosity's detail-level;
-  ``log``: log-files' detail-level (if exists).

In [49]:
for isa_type, isa_pars in pars.items():
    isa = isa_type(pi=pi, **isa_pars, seed=seed, device=device)
    # n_iter*pop_size if isinstance(isa, RandomSearch) else n_iter  # equivalency for the RS
    isa.solve(n_iter=n_iter, tol=0.1, n_iter_tol=5, verbose=2, log=0)
    print("Algorithm: {}".format(isa_type.__name__))
    print("Best solution's fitness: {:.3f}".format(isa.best_sol.fit))
    print("Best solution:", isa.best_sol.repr_, end="\n\n")

-------------------------------------------------
           |           Best solution            |
-------------------------------------------------
Generation   Length   Fitness              Timing
0            2        18.048                0.002
1            2        18.048                0.000
2            2        18.048                0.000
3            2        18.048                0.000
4            2        18.048                0.000
5            2        18.048                0.000
Algorithm: RandomSearch
Best solution's fitness: 18.048
Best solution: tensor([-0.0383,  2.7466])

--------------------------------------------------------------------------------------
           |           Best solution              |           Neighborhood           |
--------------------------------------------------------------------------------------
Generation | Length   Fitness              Timing | AVG Fitness           STD Fitness
0          | 2        18.048                0.000 | -1